In [6]:
import torch
from datetime import datetime, timedelta
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

import sys
sys.path.append('..')

from utils.plotting import plot_single_forecast
from xPatch_repo.exp.exp_main import Exp_Main

In [7]:
model = torch.load('best_model_full.pth', weights_only=False)

In [8]:
data_dir = '../../data/panama-electricity-load-forecasting/processed'

class Args:
    def __init__(self):
        self.seq_len = 24 * 4 * 4
        self.label_len = 6 * 4 # 24 * 4
        self.pred_len = 6 * 4 # 24 * 4
        
        self.is_training = 1
        self.model_id = 'test'
        self.model = 'xPatch'
        self.data = 'custom'
        
        self.root_path = data_dir
        self.data_path = "xPatch_concated_data.csv"
        
        self.features = 'MS'
        self.target = 'nat_demand'
        self.freq = 'h'
        self.scale = True
        self.timeenc = 0
        self.train_only = False
        
        self.enc_in = 13
        self.embed = 'timeF'
        
        self.patch_len = 16
        self.stride = 8
        self.padding_patch = 'end'
        
        self.ma_type = 'ema'
        self.alpha = 0.3
        self.beta = 0.3
        
        self.batch_size = 32
        self.train_epochs = 100
        self.patience = 10
        self.learning_rate = 0.0001
        self.loss = 'mse'
        self.lradj = 'type1'
        self.use_amp = False
        self.revin = 1
        
        self.use_gpu = True if torch.cuda.is_available() else False
        self.gpu = 0
        self.use_multi_gpu = False
        self.devices = '0'
        self.test_flop = False
        
        self.checkpoints = './checkpoints/'
        self.des = 'test'
        self.itr = 1
        self.num_workers = 10

        self.train_perc = 0.9
        self.test_perc = 0.05

args = Args()
experiment = Exp_Main(args)

Use GPU: cuda:0


In [9]:
test_dataset, test_loader = experiment._get_data(flag='test')

test 2202


In [10]:
all_preds = []
all_true = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for i in tqdm(range(0, len(test_dataset), args.pred_len)):
    batch_x, batch_y, _, _ = test_dataset[i]
    batch_x = torch.tensor(batch_x).unsqueeze(0).float().to(device)
    batch_y = torch.tensor(batch_y).unsqueeze(0).float().to(device)

    model.eval()
    with torch.no_grad():
        preds = model(batch_x)

    y_pred = test_dataset.inverse_transform(preds.detach().squeeze(0).cpu().numpy())[..., -1:].flatten()
    y_true = test_dataset.inverse_transform(batch_y.detach().squeeze(0).cpu().numpy())[:preds.shape[1], -1:].flatten()

    all_preds.append(y_pred)
    all_true.append(y_true)

y_pred = np.concatenate(all_preds)
y_true = np.concatenate(all_true)

  0%|          | 0/92 [00:00<?, ?it/s]

In [16]:
end_date = datetime(2020, 1, 30)
n_hours = y_pred.shape[0]

date_range = [end_date - timedelta(hours=i) for i in range(n_hours)][::-1]

In [18]:
results_df = pd.DataFrame()

results_df['data'] = date_range
results_df['y_pred'] = y_pred
results_df['y_true'] = y_true
actual_col = 'y_pred'
pred_col = 'y_true'
timestamp_col = 'data'
model_name = 'xPatch'

height = 600
width = 2000

plot_single_forecast(results_df, timestamp_col=timestamp_col, actual_col=actual_col, pred_col=pred_col, model_name=model_name, height=height, width=width, title = 'xPatch predictions')

In [13]:
results_df

,data,y_pred,y_true
0,2020-01-01 00:00:00,1111.859009,1123.479248
1,2020-01-01 01:00:00,1277.242188,1278.860352
2,2020-01-01 02:00:00,1358.454590,1372.177979
3,2020-01-01 03:00:00,1431.391846,1446.517578
4,2020-01-01 04:00:00,1445.088989,1475.093018
...,...,...,...
2203,2020-04-01 19:00:00,1051.619995,999.278931
2204,2020-04-01 20:00:00,1035.194946,988.248718
2205,2020-04-01 21:00:00,1040.834717,975.757324
2206,2020-04-01 22:00:00,1017.092834,979.774475
